# Непараметрическая статистика

---

В случае малого объёма выборки, особенности переменной или несоответствия её закона распределения нормальному, применяются методы **непараметрического анализа**. Проверим, существует ли зависимость между результатом предыдущего экзамена и итоговым баллом. В представленных ниже таблицах вычислены коэффициенты корреляции Спирмена, Тау Кендалла и Гамма. Из таблиц видно, что между переменными Рез. пр. экз и Итоговый балл корреляция слабо положительная и статистически значимая.

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr, kendalltau, norm

df = pd.read_csv("../data/student_perfomance_sample.csv")
x = df["prev_exam_score"].dropna()
y = df["final_exam_score"].dropna()

def make_corr_table(method_name, corr, pval):
    table = pd.DataFrame([[1.00, corr], [corr, 1.00]], index=["Рез. пр. экз.", "Итоговый балл"], columns=["Рез. пр. экз.", "Итоговый балл"])
    def highlight(val):
        if pd.isna(val):
            return ""
        if val == 1.00:
            return ""
        return "color: red; font-weight: bold" if pval < 0.05 else ""
    styled = table.style.map(highlight)
    print(f"\n {method_name}")
    display(styled)

def goodman_kruskal_gamma(x, y):
    x = np.array(x)
    y = np.array(y)
    n = len(x)
    concordant = 0
    discordant = 0
    for i in range(n):
        for j in range(i+1, n):
            dx = x[i] - x[j]
            dy = y[i] - y[j]
            prod = dx * dy
            if prod > 0:
                concordant += 1
            elif prod < 0:
                discordant += 1
    gamma = (concordant - discordant) / (concordant + discordant)
    se = 2 * np.sqrt((concordant * discordant) / ((concordant + discordant)**3)) if (concordant + discordant) != 0 else 1
    z = gamma / se if se != 0 else 0
    p_val = 2 * (1 - norm.cdf(abs(z)))
    return gamma, p_val

corr_s, p_s = spearmanr(x, y)
make_corr_table("Корреляция Спирмена", corr_s, p_s)

corr_k, p_k = kendalltau(x, y)
make_corr_table("Тау корреляция Кендалла", corr_k, p_k)

corr_g, p_g = goodman_kruskal_gamma(df["prev_exam_score"], df["final_exam_score"])
make_corr_table("Гамма корреляция", corr_g, p_g)


 Корреляция Спирмена


,Рез. пр. экз.,Итоговый балл
Рез. пр. экз.,1.000000,0.160801
Итоговый балл,0.160801,1.000000



 Тау корреляция Кендалла


,Рез. пр. экз.,Итоговый балл
Рез. пр. экз.,1.000000,0.112906
Итоговый балл,0.112906,1.000000



 Гамма корреляция


,Рез. пр. экз.,Итоговый балл
Рез. пр. экз.,1.000000,0.118920
Итоговый балл,0.118920,1.000000


Предположим теперь, что для качества преподавания задан порядок посредством кодов: Medium лучше Low, High лучше Medium. Тогда представленные ниже таблицы демонстрируют слабо положительную корреляцию качества преподавания с итоговым баллом, и слабо отрицательную – с результатом предыдущего экзамена, то есть с ростом качества преподавания увеличивается итоговый балл и уменьшается результат предыдущего экзамена, однако данные выводы не являются статистически значимыми.

In [2]:
mapping = {"Low": 1, "Medium": 2, "High": 3}
df["_tq_num"] = df["teaching_quality"].map(mapping)
vars_list = ["prev_exam_score", "final_exam_score"]
row_labels = ["Рез. пред. экзамена", "Итоговый балл"]
col_label = ["Качество преподавания"]

def goodman_kruskal_gamma(x, y):
    x = np.array(x)
    y = np.array(y)
    n = len(x)
    concordant = 0
    discordant = 0
    for i in range(n):
        for j in range(i+1, n):
            dx = x[i] - x[j]
            dy = y[i] - y[j]
            prod = dx * dy
            if prod > 0:
                concordant += 1
            elif prod < 0:
                discordant += 1
    gamma = (concordant - discordant) / (concordant + discordant)
    se = 2 * np.sqrt((concordant * discordant) / ((concordant + discordant)**3)) if (concordant + discordant) != 0 else 1
    z = gamma / se if se != 0 else 0
    p_val = 2 * (1 - norm.cdf(abs(z)))
    return gamma, p_val
    
def corr_table(method="spearman"):
    mat = pd.DataFrame(index=row_labels, columns=col_label, dtype=float)
    pvals = pd.DataFrame(index=row_labels, columns=col_label, dtype=float)
    for v, label in zip(vars_list, row_labels):
        x = df[v]
        y = df["_tq_num"]
        if method == "spearman":
            corr, p = spearmanr(x, y, nan_policy="omit")
        elif method == "kendall":
            corr, p = kendalltau(x, y, nan_policy="omit")
        elif method == "gamma":
            corr, p = goodman_kruskal_gamma(x, y)
        mat.loc[label, "Качество преподавания"] = corr
        pvals.loc[label, "Качество преподавания"] = p

    def highlight(val, p):
        if pd.isna(val):
            return ""
        if not pd.isna(p) and p < 0.05:
            return "color: red; font-weight: bold"
        return ""

    styled = mat.style.apply(lambda row: [highlight(row[col], pvals.loc[row.name, col]) for col in mat.columns], axis=1).format("{:.3f}")
    return styled

print("Корреляция Спирмена")
display(corr_table("spearman"))
print("Тау корреляция Кендалла")
display(corr_table("kendall"))
print("Гамма корреляция")
display(corr_table("gamma"))

Корреляция Спирмена


,Качество преподавания
Рез. пред. экзамена,-0.015
Итоговый балл,0.045


Тау корреляция Кендалла


,Качество преподавания
Рез. пред. экзамена,-0.012
Итоговый балл,0.037


Гамма корреляция


,Качество преподавания
Рез. пред. экзамена,-0.017
Итоговый балл,0.052


Теперь сравним средние в группах по переменной Тип школы. Для этого применим критерии Вальда-Вольфовица, Колмогорова-Смирнова и Манна-Уитни. Результаты представлены в таблицах ниже. По таблицам видно, что для переменной Итоговый балл верна гипотеза о равенстве средних, а для Результат предыдущего экзамена статистическая значимость средних по группам с высокой вероятностью отклоняется.

In [3]:
def wald_wolfowitz(x1, x2, continuity=False):
    a = pd.DataFrame({'v': x1, 'g': 1})
    b = pd.DataFrame({'v': x2, 'g': 0})
    pooled = pd.concat([a, b], ignore_index=True)
    pooled = pooled.sort_values(by=['v']).reset_index(drop=True)
    labels = pooled['g'].values
    runs = 1 + np.sum(labels[1:] != labels[:-1])
    n1 = int(labels.sum())
    n2 = int(len(labels) - n1)
    expected = 1 + (2.0 * n1 * n2) / (n1 + n2)
    var = (2.0 * n1 * n2 * (2.0 * n1 * n2 - n1 - n2)) / ((n1 + n2)**2 * (n1 + n2 - 1.0))
    
    if continuity: cc = 0.5 if runs > expected else -0.5
    else: cc = 0.0
    
    z = (runs - expected - cc) / np.sqrt(var) if var > 0 else np.nan
    p = 2.0 * (1.0 - norm.cdf(abs(z)))
    table = pd.DataFrame({
        "N (Public)": [n1],
        "N (Private)": [n2],
        "Среднее (Public)": [x1.mean()],
        "Среднее (Private)": [x2.mean()],
        "Z": [z],
        "p-value": [p],
        "Runs": [runs]
    })
    
    def highlight(val):
        if isinstance(val, (float, np.floating)) and val < 0.05:
            return "color: red; font-weight: bold"
        return ""

    return table.style.map(highlight, subset=["p-value"])

print("Тест Вальда-Вольфовица для результата предыдущего экзамена:")
display(wald_wolfowitz(df.loc[df['school_type']=='Public', 'prev_exam_score'].dropna().values, x2 = df.loc[df['school_type']=='Private', 'prev_exam_score'].dropna().values))
print("Тест Вальда-Вольфовица для итогового балла:")
display(wald_wolfowitz(df.loc[df['school_type']=='Public', 'final_exam_score'].dropna().values, df.loc[df['school_type']=='Private', 'final_exam_score'].dropna().values))

Тест Вальда-Вольфовица для результата предыдущего экзамена:


,N (Public),N (Private),Среднее (Public),Среднее (Private),Z,p-value,Runs
0,667,277,74.239880,75.678700,-2.155306,0.031138,365


Тест Вальда-Вольфовица для итогового балла:


,N (Public),N (Private),Среднее (Public),Среднее (Private),Z,p-value,Runs
0,667,277,67.223388,67.144404,-2.548058,0.010832,360


In [4]:
from scipy.stats import ks_2samp

def ks_table(df, variable, group_var="school_type"):
    groups = df[group_var].dropna().unique()
    g1, g2 = groups
    x1 = df[df[group_var] == g1][variable].dropna()
    x2 = df[df[group_var] == g2][variable].dropna()
    ks_res = ks_2samp(x1, x2, alternative="two-sided")
    table = pd.DataFrame({
        "N (Public)": [len(x1)],
        "N (Private)": [len(x2)],
        "Среднее (Public)": [x1.mean()],
        "Среднее (Private)": [x2.mean()],
        "D": [ks_res.statistic],
        "p-value": [ks_res.pvalue]
    })

    def highlight(val):
        if isinstance(val, (float, np.floating)) and val < 0.05:
            return "color: red; font-weight: bold"
        return ""

    return table.style.map(highlight, subset=["p-value"])

print("Тест Колмогорова-Смирнова для результата предыдущего экзамена:")
display(ks_table(df, "prev_exam_score"))
print("Тест Колмогорова-Смирнова для итогового балла:")
display(ks_table(df, "final_exam_score"))

Тест Колмогорова-Смирнова для результата предыдущего экзамена:


,N (Public),N (Private),Среднее (Public),Среднее (Private),D,p-value
0,667,277,74.239880,75.678700,0.057713,0.509224


Тест Колмогорова-Смирнова для итогового балла:


,N (Public),N (Private),Среднее (Public),Среднее (Private),D,p-value
0,667,277,67.223388,67.144404,0.033687,0.972543


In [5]:
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu, rankdata

def mann_whitney_table(df, variable):
    g1 = df[df["school_type"] == "Public"][variable].dropna()
    g2 = df[df["school_type"] == "Private"][variable].dropna()
    N1, N2 = len(g1), len(g2)
    all_values = np.concatenate([g1, g2])
    ranks = rankdata(all_values)
    rank_sum1 = ranks[:N1].sum()
    rank_sum2 = ranks[N1:].sum()
    
    U1, p = mannwhitneyu(g1, g2, alternative="two-sided")

    mu_U = N1 * N2 / 2
    sigma_U = np.sqrt(N1 * N2 * (N1 + N2 + 1) / 12)
    Z = (U1 - mu_U) / sigma_U

    table = pd.DataFrame({
        "Сумма рангов (Public)": [rank_sum1],
        "Сумма рангов (Private)": [rank_sum2],
        "U": [U1],
        "Z": [Z],
        "p-value": [p],
        "N (Public)": [N1],
        "N (Private)": [N2]
    })

    def highlight(val):
        if isinstance(val, float) and val < 0.05:
            return "color: red; font-weight: bold"
        return ""

    styled = table.style.map(highlight, subset=["p-value"])
    return styled

print("U критерий Манна-Уитни для результата предыдущего экзамена:")
display(mann_whitney_table(df, "prev_exam_score"))
print("U критерий Манна-Уитни для итогового балла:")
display(mann_whitney_table(df, "final_exam_score"))

U критерий Манна-Уитни для результата предыдущего экзамена:


,Сумма рангов (Public),Сумма рангов (Private),U,Z,p-value,N (Public),N (Private)
0,309900.000000,136140.000000,87122.000000,-1.378324,0.168046,667,277


U критерий Манна-Уитни для итогового балла:


,Сумма рангов (Public),Сумма рангов (Private),U,Z,p-value,N (Public),N (Private)
0,316044.500000,129995.500000,93266.500000,0.232539,0.815544,667,277


Далее для итогового балла проведём анализ, разбивая выборку на более чем две группы – по качеству преподавания. Данные, представленные в таблицах ниже, позволяют судить о равенстве средних как во всех 3 группах в целом, так и в каждой их паре.

In [6]:
from scipy.stats import kruskal
from scikit_posthocs import posthoc_dunn

order = ["Low", "Medium", "High"]
groups = [df[df["teaching_quality"] == g]["final_exam_score"].dropna() for g in order]
all_data = df[["teaching_quality", "final_exam_score"]].dropna()
all_data["rank"] = all_data["final_exam_score"].rank()

rows = []
for i, g in enumerate(order, start=1):
    grp = all_data[all_data["teaching_quality"] == g]
    n = len(grp)
    sum_ranks = grp["rank"].sum()
    mean_rank = sum_ranks / n
    rows.append([g, i, n, sum_ranks, mean_rank])

table_kw = pd.DataFrame(rows, columns=["Группа", "Код", "N", "Сумма рангов", "Среднее ранг"])
kw_stat, kw_p = kruskal(*groups)
print("p-значение Краскела–Уоллиса:", kw_p)
display(table_kw)

posthoc = posthoc_dunn(all_data, val_col="final_exam_score", group_col="teaching_quality", p_adjust="holm")
print("p-значения (двусторонние) для множественных сравнений:")
display(posthoc.loc[order, order])

p-значение Краскела–Уоллиса: 0.08220680584907991


,Группа,Код,N,Сумма рангов,Среднее ранг
0,Low,1,91,37496.0,412.043956
1,Medium,2,564,269781.5,478.335993
2,High,3,289,138762.5,480.147059


p-значения (двусторонние) для множественных сравнений:


,Low,Medium,High
Low,1.000000,0.092246,0.092246
Medium,0.092246,1.000000,0.926567
High,0.092246,0.926567,1.000000


Для анализа *повторных измерений* (результат предыдущего экзамена и итоговый балл) применим критерии знаков и Вилкоксона. Представленные ниже таблицы демонстрируют статистическую значимость отличия средних при повторном измерении, причем средний результат предыдущего экзамена статистически значимо больше среднего итогового балла.

In [7]:
from scipy.stats import binomtest, wilcoxon

def sign_test_table(df):
    data = df[["prev_exam_score", "final_exam_score"]].dropna()
    a = data["prev_exam_score"]
    b = data["final_exam_score"]

    diff = b - a
    diff_nonzero = diff[diff != 0]
    n = len(diff_nonzero)

    v = (diff_nonzero > 0).sum()
    V = min(v, n - v)

    p_val = binomtest(v, n, p=0.5, alternative="two-sided").pvalue
    Z = (abs(v - n/2) - 0.5) / np.sqrt(n/4)

    table = pd.DataFrame([{"Число несовпадений": n, "Процент v < V": round(100 * V / n, 2), "Z": Z, "p_value": p_val}], index=["Рез. пр. экз & Итоговый балл"])
    def highlight(row):
        return ["color: red; font-weight: bold" if row["p_value"] < 0.05 else "" for _ in row]
    return table.style.apply(highlight, axis=1)

def wilcoxon_table(df):
    data = df[["prev_exam_score", "final_exam_score"]].dropna()
    a = data["prev_exam_score"]
    b = data["final_exam_score"]

    res = wilcoxon(b, a, zero_method="wilcox", alternative="two-sided")

    diff = b - a
    diff_nonzero = diff[diff != 0]
    n = len(diff_nonzero)

    T = res.statistic
    mean_T = n*(n+1)/4
    sd_T = np.sqrt(n*(n+1)*(2*n+1)/24)
    Z = (T - mean_T)/sd_T

    table = pd.DataFrame([{"Число наблюдений": n, "T": T, "Z": Z, "p_value": res.pvalue}], index=["Рез. пр. экз & Итоговый балл"])
    def highlight(row):
        return ["color: red; font-weight: bold" if row["p_value"] < 0.05 else "" for _ in row]
    return table.style.apply(highlight, axis=1)

display(sign_test_table(df))
display(wilcoxon_table(df))

,Число несовпадений,Процент v < V,Z,p_value
Рез. пр. экз & Итоговый балл,920,33.910000,9.725862,0.000000


,Число наблюдений,T,Z,p_value
Рез. пр. экз & Итоговый балл,920,100291.000000,-13.835082,0.000000
